In [1]:
import numpy as np, pandas as pd, SimpleITK as sitk, matplotlib.pyplot as plt, os
import tensorflow as tf, pydicom as dicom
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
#GCS_PATH = KaggleDatasets().get_gcs_path()
TRAIN_IMG_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'
#BATCH_SIZE = 16 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]
EPOCHS = 25
TARGET_COLS  = [
        "bowel_injury", "extravasation_injury",
        "kidney_healthy", "kidney_low", "kidney_high",
        "liver_healthy", "liver_low", "liver_high",
        "spleen_healthy", "spleen_low", "spleen_high",
    ]

In [3]:
train = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv')
train = train.head(50)
display(train)

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury
0,10004,1,0,0,1,0,1,0,1,0,0,0,0,1,1
1,10005,1,0,1,0,1,0,0,1,0,0,1,0,0,0
2,10007,1,0,1,0,1,0,0,1,0,0,1,0,0,0
3,10026,1,0,1,0,1,0,0,1,0,0,1,0,0,0
4,10051,1,0,1,0,1,0,0,1,0,0,0,1,0,1
5,10065,0,1,1,0,1,0,0,1,0,0,1,0,0,1
6,10082,1,0,1,0,1,0,0,1,0,0,1,0,0,0
7,10104,1,0,1,0,1,0,0,1,0,0,1,0,0,0
8,10105,1,0,1,0,1,0,0,1,0,0,1,0,0,0
9,10124,1,0,1,0,1,0,0,1,0,0,1,0,0,0


In [4]:
# A path to a T1-weighted brain .nii image:
segmentation_dir = '/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/'
train_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images'

segs = os.listdir(segmentation_dir)
trains = os.listdir(train_path)
train['patient_id'] = train['patient_id'].astype(str)
X_train, X_valid, y_train, y_valid = train_test_split(train.patient_id, train.drop(['patient_id'], axis = 1))

In [5]:
y_train['patient_id'] = X_train
y_valid['patient_id'] = X_valid
train_dataset = y_train
valid_dataset = y_valid
display(train_dataset)

,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,liver_high,spleen_healthy,spleen_low,spleen_high,any_injury,patient_id
44,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10762
5,0,1,1,0,1,0,0,1,0,0,1,0,0,1,10065
18,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10232
45,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10795
12,1,0,1,0,1,0,0,0,1,0,1,0,0,1,10163
20,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1027
14,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10172
35,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10569
33,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10557
8,1,0,1,0,1,0,0,1,0,0,1,0,0,0,10105


In [6]:
def save_img(patient_id, output_path):
    for directory in os.listdir(os.path.join(TRAIN_IMG_PATH, patient_id)):
        files = os.listdir(os.path.join(TRAIN_IMG_PATH, patient_id, directory))
        files.sort(key = lambda x: int(x.split('.')[0]))
        files = files[::int(len(files)//4)]
        files = [os.path.join(TRAIN_IMG_PATH, patient_id, directory, x) for x in files]
        img_arrays = []
        for img in files:
            ds = dicom.dcmread(img)
            img_arrays.append(ds.pixel_array)
        img_3d = np.array(img_arrays)
    np.save(os.path.join(output_path,str(patient_id)+'.npy'), img_3d)

def save_img_to_numpy(dataset, output_path):
    #print(dataset)
    for index, row in dataset.iterrows():
        save_img(row['patient_id'], output_path)

if not os.path.isdir('train_images'): os.mkdir('train_images')
if not os.path.isdir('valid_images'): os.mkdir('valid_images')

#print(type(train_dataset))
save_img_to_numpy(train_dataset, 'train_images')
save_img_to_numpy(valid_dataset, 'valid_images')

# Credits

This notebook was forked from https://www.kaggle.com/code/awsaf49/rsna-atd-cnn-tpu-train

Evaluation metric from https://www.kaggle.com/code/metric/rsna-trauma-metric/notebook